First run the installs:

In [ ]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64
from bs4 import BeautifulSoup
import pandas as pd

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

def get_email_content(payload):
    if 'parts' in payload:
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain':
                return base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
            elif part['mimeType'] == 'text/html':
                html_content = base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
                return BeautifulSoup(html_content, "html.parser").get_text()
    elif 'body' in payload:
        return base64.urlsafe_b64decode(payload['body']['data']).decode('utf-8')
    return "No content available"

def get_latest_emails(service, num_emails=10):
    results = service.users().messages().list(userId='me', maxResults=num_emails, q="is:inbox").execute()
    messages = results.get('messages', [])

    email_data = []
    for msg in messages:
        message = service.users().messages().get(userId='me', id=msg['id'], format='full').execute()

        headers = message['payload']['headers']
        date = next((h['value'] for h in headers if h['name'] == 'Date'), "No Date")
        sender = next((h['value'] for h in headers if h['name'] == 'From'), "Unknown Sender")
        subject = next((h['value'] for h in headers if h['name'] == 'Subject'), "No Subject")

        # Parse sender's email and name
        sender_name = sender
        sender_email = sender
        if "<" in sender and ">" in sender:
            sender_name, sender_email = sender.split(" <", 1)
            sender_email = sender_email.strip(">")

        # Get the email content
        email_content = get_email_content(message['payload'])

        # Append data to list
        email_data.append([date, sender_email, sender_name, subject, email_content])

    # Create DataFrame
    df = pd.DataFrame(email_data, columns=["Date Received", "Sender Email", "Sender Name", "Subject", "Message Content"])
    return df

def main():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        # Call the Gmail API
        service = build("gmail", "v1", credentials=creds)
        
        # Fetch the latest 10 emails in a DataFrame
        df = get_latest_emails(service, num_emails=10)
        print(df)

        return df

    except HttpError as error:
        print(f"An error occurred: {error}")

if __name__ == "__main__":
    main()
